## Using Keras Sequential Model and Functional API

In [ ]:
import os, random, numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

%matplotlib inline

np.set_printoptions(precision=2, suppress=True)

In [ ]:
def softmax(logits):
    exp_logits = np.exp(logits)
    return exp_logits / np.sum(exp_logits)

logits = np.array([2.0, 1.0, 0.1])
probabilities = softmax(logits)
print(probabilities, sum(probabilities))

In [ ]:
# Load MNIST data

seed_value = 1234

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

# Sequential Model

In [ ]:
# Sequential Model (Very convenient, not very flexible)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

model_1 = keras.Sequential(
    [
        keras.Input(shape=(x_train.shape[-1],)),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(10),
    ]
)

model_1.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model_1.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_1.evaluate(x_test, y_test, batch_size=32, verbose=2)))

In [ ]:
# Another way to use Sequential Model

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

model_1b = keras.Sequential()
model_1b.add(keras.Input(shape=(x_train.shape[-1],)))
model_1b.add(layers.Dense(512, activation="relu", name="fc_layer1"))
model_1b.add(layers.Dense(256, activation="relu", name="fc_layer2"))
model_1b.add(layers.Dense(10, name="outputs"))

model_1b.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

model_1b.summary()

In [ ]:
model_1b.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_1b.evaluate(x_test, y_test, batch_size=32, verbose=2)))

## Categorical Crossentropy

In [ ]:
from keras.utils import to_categorical

y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

model_1c = keras.Sequential()
model_1c.add(keras.Input(shape=(x_train.shape[-1],)))
model_1c.add(layers.Dense(512, activation="relu", name="fc_layer1"))
model_1c.add(layers.Dense(256, activation="relu", name="fc_layer2"))
model_1c.add(layers.Dense(10, activation="softmax", name="outputs"))

model_1c.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

model_1c.summary()

model_1c.fit(x_train, y_train_one_hot, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_1c.evaluate(x_test, y_test_one_hot, batch_size=32, verbose=2)))

In [ ]:
y_pred = model_1c.predict(x_test)

In [ ]:
y_pred.argmax(axis=1)

In [ ]:
y_test

In [ ]:
print((y_test == y_pred.argmax(axis=1)).mean())

# Functional API

In [ ]:
# Functional API (A bit more flexible)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

inputs = keras.Input(shape=(x_train.shape[-1],), name="inputs")
x = layers.Dense(512, activation="relu", name="fc_layer1")(inputs)
x = layers.Dense(256, activation="relu", name="fc_layer2")(x)
outputs = layers.Dense(10, activation="softmax", name="outputs")(x)

model_2 = keras.Model(inputs=inputs, outputs=outputs)

model_2.summary()

model_2.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model_2.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_2.evaluate(x_test, y_test, batch_size=32, verbose=2)))

# More on Functional API

In [ ]:
# Functional API (A bit more flexible)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

inputs = keras.Input(shape=(x_train.shape[-1],), name="inputs")
x1 = layers.Dense(512, activation="relu", name="fc_layer1")(inputs)
x2 = layers.Dense(256, activation="relu", name="fc_layer2")(x1)
x = layers.Concatenate(axis=1, name="concat_layer1")([x1, x2])
x = layers.Dense(64, activation="relu", name="fc_layer3")(x)
outputs = layers.Dense(10, activation="softmax", name="outputs")(x)

model_3 = keras.Model(inputs=inputs, outputs=outputs)

model_3.summary()

model_3.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model_3.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_3.evaluate(x_test, y_test, batch_size=32, verbose=2)))